In [39]:
# 需要进行建立模型
import keras
from keras.preprocessing.image import ImageDataGenerator
# 隐藏层
from keras.layers import Conv2D, Flatten, MaxPool2D, Dense,Dropout
from keras.models import Sequential
# 优化方法
from keras.optimizers import Adam

In [40]:
train_dir = 'images'
num_epochs = 1
batch_size = 100

In [41]:
# rescale 像素比例缩放  validation_split 验证集数据比例
data_gen = ImageDataGenerator(rescale=1. / 255, validation_split=0.1)

# train_dir  文件夹
# target_size=(64, 64)  对象缩放比例
# batch_size=batch_size 一次读入多少图片，一共读多少数据，决定与data_gen中的validation_split验证集切割
# class_mode  固定字段，会根据给定路径下的文件夹做分类
# subset   training validation  训练集  和验证集 
train_generator = data_gen.flow_from_directory(train_dir,
                                               target_size=(64, 64),
                                               batch_size=batch_size,
                                               class_mode='categorical', subset='training')
validation_generator = data_gen.flow_from_directory(train_dir,
                                               target_size=(64, 64),
                                               batch_size=batch_size,
                                               class_mode='categorical', subset='validation')

Found 2276 images belonging to 7 classes.
Found 251 images belonging to 7 classes.


In [42]:
#获得原始数据
# train_generator.__getitem__(1)
#获得分类对应
labels =train_generator.class_indices
labels

{'.ipynb_checkpoints': 0,
 'cardboard': 1,
 'glass': 2,
 'metal': 3,
 'paper': 4,
 'plastic': 5,
 'trashs': 6}

In [43]:
# *******************************************************************
model=Sequential()
#输入层
model.add(Conv2D(64,3,3,input_shape=(64, 64,3),activation='relu'))
#隐藏层  计算
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(output_dim=32,activation='relu'))
# +1~-1  宽度比较大
model.add(Dense(output_dim=7,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 31, 31, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 61504)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1968160   
_________________________________________________________________
dense_8 (Dense)              (None, 7)                 231       
Total params: 1,970,183
Trainable params: 1,970,183
Non-trainable params: 0
_________________________________________________________________


/home/baicaitong/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
  after removing the cwd from sys.path.
/home/baicaitong/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=32)`
  if __name__ == '__main__':
/home/baicaitong/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=7)`
  # This is added back by InteractiveShellApp.init_path()


In [44]:
#损失函数
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
# 试试看 3轮 准确率 88.75%
# model.fit(X_train , y_train, batch_size= 15, epochs = 3)
# *******************************************************************
# steps_per_epoch  一次训练多少张
# validation_steps  一次验证多少张
# epochs  训练几轮
#validation_data 验证数据是什么

In [34]:
model.fit_generator(train_generator, 
                    steps_per_epoch= 5, #100
                    validation_steps=1, #50
                    epochs= 3, 
                    validation_data=validation_generator)

#acc是准确度  loss是错误

Epoch 1/3
5/5 [==============================] - 3s 674ms/step - loss: 2.1435 - acc: 0.2380 - val_loss: 1.7141 - val_acc: 0.1700
Epoch 2/3
5/5 [==============================] - 3s 629ms/step - loss: 1.7744 - acc: 0.2300 - val_loss: 1.7155 - val_acc: 0.2500
Epoch 3/3
5/5 [==============================] - 4s 752ms/step - loss: 1.7638 - acc: 0.2160 - val_loss: 1.6675 - val_acc: 0.3333


---

In [35]:
model.save('my_model1.h5')

In [36]:
# 加载模型
from keras.models import load_model
model = load_model('my_model1.h5')

In [45]:
from PIL import Image
import numpy as np
image = Image.open("test.jpg") # 用PIL中的Image.open打开图像
img2=image.resize((64, 64),Image.ANTIALIAS)
image_arr = np.array(img2) # 转化成numpy数组
image_arr=image_arr/255
image_arr.shape
image_1=np.array([image_arr])

# test_x, test_y = validation_generator.__getitem__(1)
# labels = (train_generator.class_indices)
# labels = dict((v,k) for k,v in labels.items())
# preds = model.predict(test_x)
# preds
# plt.figure(figsize=(16, 16))
# for i in range(16):
#     plt.subplot(4, 4, i+1)
#     plt.title('pred:%s / truth:%s' % (labels[np.argmax(preds[i])], labels[np.argmax(test_y[i])]))
#     plt.imshow(test_x[i])

#预测一张怎么预测
#导入导出模型 深度学习h5 机器学习pkl


In [46]:
preds = model.predict(image_1)
preds

array([[0.5145442 , 0.5461345 , 0.5210257 , 0.33930483, 0.45196882,
        0.57053345, 0.35542372]], dtype=float32)